# Let's biuld SVM for binary classification and use quadratic optimization package CVXOPT in python.

- SVM supports regression too. Look into literature for regression usage
- SVM supports multiclass classification too
- We can combine muliple kernel to use data from different sources(video, image ,audio) and using right kernel to measure similarity and then combine.  See MKL https://en.wikipedia.org/wiki/Multiple_kernel_learning 
  
- **One class SVM** are used for novelty (anamolies, outlier, noise etc) detection in fraud detection, medical abnormality, production abnormality etc. One can also use **ensemble.IsolationForest** from ensemble methods.
- See this paper for a review of novelty detection methods. http://www.robots.ox.ac.uk/~davidc/pubs/NDreview2014.pdf

# Let's review some theory.

In this notebook we will implement  SVM. We know that SMV primal objective is

$\min_{w, w_0} \frac{1}{2}||w||^2 + C \sum_{i=1}^{N} \xi_i$ st $y_i(w^T +w_0) \gt 1 $  for $\forall i$

Note the label is $(+1, -1)$ instead of $(0, 1)$


Infact Dual problem(using Lagrangian) is written as 
$\min_{\alpha} \frac{1}{2}\sum_{i=1}^N \alpha_i \alpha_j y_i y_j(x_i^T x_j) - 1^T \alpha$ s.t $ \alpha_i \ge 0$ and $y^T\alpha = 0$

or 

$\min_{\alpha} \frac{1}{2}\alpha^T K \alpha - 1^T \alpha$ s.t $ \alpha_i \ge 0$ and $y^T\alpha = 0$

where  $\frac{1}{2}\alpha^T K \alpha$ = $\sum_{i=1}^N \alpha_i \alpha_j y_i y_j(x_i^T x_j)$ and $\alpha$ is vector of $\alpha_i$. Similar interpretation for $y$ and $y_i$. 


optimization theory derivation here http://cs229.stanford.edu/notes/cs229-notes3.pdf


dual problem depends only on innner product $x_i^Tx_j$ between $x_i, x_j$ . Hence we can replace it with a mercer kernal $\mathcal{k}(x_i, x_j) = \phi(x_i)^T \phi(x_j)$ where $\phi$ is the function
guranteed by mercer theorem. You can think of it as a feature mapping function $x_i \rightarrow \phi(x_i)$.
For kernel svm replace matrix K  using RBF kernel i.e $K_{ij} = y_i  y_j \mathcal{k}(x_i,x_j)$ where  $\mathcal{k}$ is any kernel. **Use RBF kernel for this assignment.**

Complete dual problem in dual form is
$\min_{\alpha} \frac{1}{2}\sum_{i=1}^N \alpha_i \alpha_j y_i y_j \mathcal{k}(x_i, x_j) - 1^T \alpha$ s.t $ \alpha_i \ge 0$ and $y^T\alpha = 0$




We will use cvxopt Quadratic program solver (QP).

cvxopt.solvers.qp(P, q[, G, h[, A, b[, solver[, initvals]]]])

which solves the problem


$\min_{x} \frac{1}{2}x^T P x {\bf+} q^T x$ s.t $Gx \preceq h$ and $Ax = b$. Note $\preceq$ denotes componentwise inequality


Hence we need to map to this interface. Clearly $x$ is $\alpha$ and $K$ is $P$, $q$ is vector of -1.

Now we have to build  inequality

**(i)** $\alpha_i \ge 0$ or $ -\alpha_i < 0 $ for all $i$ or in matrix 
form $- I\alpha \preceq {\bf 0}$ where $I$ is  identity matrix of size $N \times N$ and  ${\bf 0}$ is vector of zeros of size N(number of samples)




 
Hence $G $ will be $\begin{bmatrix} -I  \end{bmatrix}$  and $h$ is $\begin{bmatrix} {\bf 0}   \end{bmatrix} $


$A$  is $y^T$ and $b$ is $0$

Once we solve for $\alpha$, w is equal to $\sum_{i} \alpha_i y_i \phi(x_i)$ and $w_0$  is equal to $\frac{max_{j:y_j =-1} \sum_{i} \alpha_i y_i k(x_i, x_j) +  min_{j:y_j =1} \sum_{i} \alpha_i y_i k(x_i, x_j)}{2}$ 

We can use the sign$(w^T x_{test} + w_0)$  or  sign$(\sum_{i}^{N} \alpha_i  y_i \mathcal{k}(x_i,x_{test}) + w_0)$ to predict label of test data



# Let's install  cvxopt package for optimization 


In [1]:
import os

In [5]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import cvxopt

This helped me learn more about [Python Software for Convex Optimization](http://cvxopt.org/)

In [6]:
data = load_breast_cancer()

In [7]:
list(data.target_names)

['malignant', 'benign']

In [8]:
len(data.feature_names)

30

In [9]:
X = data.data
y= data.target

In [10]:

y[0:30]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0])

## Let's convert dataset label to {-1, +1}
We need to convert 0 to -1 only

In [11]:
y1= np.where(y==0, -1, y)

In [12]:
y1[0:30]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1])

In [13]:
sum(y1 ==1), sum(y1 ==-1)

(357, 212)

## We are ignoring data imbalance issue

In [14]:
y = y1

In [15]:
np.max(X, axis=0)

array([2.811e+01, 3.928e+01, 1.885e+02, 2.501e+03, 1.634e-01, 3.454e-01,
       4.268e-01, 2.012e-01, 3.040e-01, 9.744e-02, 2.873e+00, 4.885e+00,
       2.198e+01, 5.422e+02, 3.113e-02, 1.354e-01, 3.960e-01, 5.279e-02,
       7.895e-02, 2.984e-02, 3.604e+01, 4.954e+01, 2.512e+02, 4.254e+03,
       2.226e-01, 1.058e+00, 1.252e+00, 2.910e-01, 6.638e-01, 2.075e-01])

# Let's standardize the features

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()
scaler.fit(X)
X1 = scaler.transform(X)

In [18]:
np.max(X1, axis=0)

array([ 3.97128765,  4.65188898,  3.97612984,  5.25052883,  4.77091122,
        4.56842498,  4.24358882,  3.92792966,  4.48475086,  4.91091929,
        8.90690934,  6.65527935,  9.46198577, 11.04184226,  8.02999927,
        6.14348219, 12.0726804 ,  6.64960079,  7.07191706,  9.85159257,
        4.09418939,  3.88590505,  4.28733746,  5.9301724 ,  3.95537411,
        5.11287727,  4.7006688 ,  2.68587702,  6.04604135,  6.84685604])

In [19]:
X= X1

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [22]:
y_train.shape, y_val_test.shape

((341,), (228,))

In [23]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=2)

In [24]:
y_val.shape, y_test.shape

((114,), (114,))

# We will Implement Kernel SVM using solver from cvxopt. Use validation set to select best model and  evaluate the model on test set


In [25]:
def  rbf_kernel (x1,x2, sigma):
    return np.exp(- np.linalg.norm(x1- x2)**2/(2.0* sigma**2))

In [26]:
# Let declare matrix K
# We need to build it different sigma
K = np.zeros((X_train.shape[0], X_train.shape[0]))
print(X_train.shape, K.shape)
n_samples = X_train.shape[0]

(341, 30) (341, 341)


In [27]:
q= cvxopt.matrix(-1*np.ones(n_samples))
q.size

(341, 1)

# Build matrix  G

In [28]:
# -I 
G = cvxopt.matrix(np.negative(np.identity(X_train.shape[0])))
G.size

(341, 341)

# Build matrix h

In [29]:
h = cvxopt.matrix(np.zeros((X_train.shape[0],1)), tc="d")
h.size

(341, 1)

## Note y_train contains integer label value (1, -1) but in cvopt library A needs to be double.

In [30]:
A = cvxopt.matrix(np.expand_dims(y_train, axis=0), tc = "d")
A.size

(1, 341)

In [31]:
b= cvxopt.matrix(0.0)

In [32]:
def predict_label(x, svec_alphas, svecs, svec_labels, wo, sigma):
    val = wo
    for alpha, svec, lbl in  zip(svec_alphas, svecs, svec_labels):
        val +=  alpha*lbl*rbf_kernel(x,svec, sigma )
        
    return np.sign(val)   



In RBF kernel $k(x_1, x_1) = \exp(\frac{\|x_1 -x_2\|^2}{-2 \sigma^2})$. Also written as  $ \exp(- \gamma \|x_1 -x_2\|^2)$ there is a hyper parameter $\sigma$ which we need to decide before solving the  problem using optimizer.

- For low value of $\sigma$ or large value of $\gamma$, area of influence is close to any support vectors. Clearly support vector influence increases as $\sigma$ increase or $\gamma$ decreases. It is similary to effect of $\sigma$ in gussian distribution.



Search for $\sigma.$ 

<font color = 'red'>Note choose right value of $\sigma$ is very important in 
RBF kernel. </font>



In [49]:
SUPPORT_VECTOR_WEIGHT_THR = 1e-4

best_val_accuracy = -np.inf
best_support_vectors_info= None
for sigma in np.linspace(.05, 15, 15):    
    #Let's iterate over sample and fill matrix K
    # Maybe we can do following operations in matrix form.
    # Going with two loops right now
    for i, x1 in enumerate(X_train):
        for j, x2 in enumerate(X_train):
            #print(x1.shape, x2.shape)
            K[i,j] = rbf_kernel(x1, x2, sigma)
    
    P = cvxopt.matrix(np.outer(y_train, y_train)*K )           
    # Let'solve the optimization problem using qudratic solver from cvxopt
    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    svec_loc = alphas > SUPPORT_VECTOR_WEIGHT_THR
    svec_alphas = alphas[svec_loc]
    
    # svecs contains our support vector
    svecs, svec_labels = X_train[ np.ravel(svec_loc),:], y_train[ np.ravel(svec_loc)]
    # evaluating the intercept for all the support vectors.
    svec_wo_pos = []
    svec_wo_neg = []
    for svec1 ,lbl1 in zip(svecs, svec_labels):
        val = 0
        for alpha, svec2, lbl2 in  zip(svec_alphas, svecs, svec_labels):
            val +=  alpha*lbl2*rbf_kernel(svec1,svec2, sigma )
        if lbl1== +1:    
            svec_wo_pos.append( val)
        elif lbl1 == -1:
            svec_wo_neg.append(val)
        else:
            print('erro support vector label in not +ve or -ve')

    #Let use first/ or mean?
    #wo = np.mean(svec_wo)
    wo = -(max(svec_wo_neg) + min(svec_wo_pos))/2.0
    
    val_acc = np.mean([ y == predict_label(x, svec_alphas, svecs, svec_labels, wo, sigma)
                       for  x, y in  zip(X_val, y_val)])
    print('sigma = {} validation accuracy = {}'.format(sigma,val_acc))
    if val_acc > best_val_accuracy:
        print('updating best model current val {} previous val {}'.format(val_acc, best_val_accuracy))
        best_support_vector_info = (sigma, svec_alphas,svecs, svec_labels, wo)
        best_val_accuracy = val_acc
        


     pcost       dcost       gap    pres   dres
 0: -1.1836e+02 -2.9770e+02  2e+02  3e-14  2e+00
 1: -1.4745e+02 -1.5011e+02  3e+00  3e-14  3e-01
 2: -1.5782e+02 -1.5821e+02  4e-01  6e-15  4e-17
 3: -1.5782e+02 -1.5782e+02  4e-03  4e-14  6e-17
 4: -1.5782e+02 -1.5782e+02  4e-05  2e-14  5e-17
Optimal solution found.
sigma = 0.05 validation accuracy = 0.5701754385964912
updating best model current val 0.5701754385964912 previous val -inf
     pcost       dcost       gap    pres   dres
 0: -8.6374e+01 -2.5994e+02  8e+02  2e+01  2e+00
 1: -9.2180e+01 -1.9196e+02  1e+02  5e-15  9e-16
 2: -1.0747e+02 -1.1639e+02  9e+00  3e-15  6e-16
 3: -1.0831e+02 -1.0944e+02  1e+00  2e-15  3e-16
 4: -1.0847e+02 -1.0857e+02  1e-01  1e-14  3e-16
 5: -1.0849e+02 -1.0850e+02  5e-03  4e-15  3e-16
 6: -1.0849e+02 -1.0850e+02  1e-04  9e-15  3e-16
 7: -1.0850e+02 -1.0850e+02  1e-06  4e-15  3e-16
Optimal solution found.
sigma = 1.1178571428571429 validation accuracy = 0.8596491228070176
updating best model current 

     pcost       dcost       gap    pres   dres
 0: -8.0760e+01 -2.3635e+02  1e+03  3e+01  2e+00
 1: -1.6144e+02 -3.9580e+02  1e+03  2e+01  2e+00
 2: -3.9631e+02 -8.0397e+02  1e+03  1e+01  1e+00
 3: -7.5453e+02 -1.2142e+03  9e+02  1e+01  9e-01
 4: -1.5040e+03 -2.0626e+03  1e+03  9e+00  7e-01
 5: -2.4338e+03 -3.2000e+03  1e+03  6e+00  5e-01
 6: -2.8793e+03 -3.0645e+03  3e+02  8e-01  7e-02
 7: -2.8718e+03 -2.8849e+03  2e+01  4e-02  4e-03
 8: -2.8736e+03 -2.8738e+03  4e-01  7e-04  6e-05
 9: -2.8736e+03 -2.8736e+03  1e-02  1e-05  1e-06
10: -2.8736e+03 -2.8736e+03  5e-04  2e-07  1e-08
11: -2.8736e+03 -2.8736e+03  5e-06  2e-09  1e-10
Optimal solution found.
sigma = 15.0 validation accuracy = 0.9473684210526315


In [34]:
# picking best sigma for accuracy on validation set 
best_sigma, svec_alphas, svecs, svec_labels, wo = best_support_vector_info

# Writing code  to evalue validation and test set accuracy

In [35]:
VAccuracy= np.mean([y== predict_label(x,svec_alphas,svecs,svec_labels,wo,best_sigma) for x,y in zip(X_val, y_val)])
print('validation accuracy = {}'.format(VAccuracy))
TAccuracy= np.mean([y== predict_label(x,svec_alphas,svecs,svec_labels,wo,best_sigma) for x,y in zip(X_test, y_test)])
print('Test accuracy = {}'.format(TAccuracy))

validation accuracy = 0.9736842105263158
Test accuracy = 0.9210526315789473


# Using sklearn kernel SVM with soft margin interface. Select best model and evaluate accuracy on test set using SVM from  the library

Check your accuracy value matches

Hint

- You need to use large value of C to simulate hard margin SVM
- gamma= 1/(2*best_sigma**2)

In [46]:
from sklearn import svm
gamma= 1/(2*best_sigma*2)
clf = svm.SVC(C= 1, gamma= gamma)
clf.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.07683863885839738,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [47]:
clf.score(X_test, y_test)

0.9473684210526315

In [38]:
clf.intercept_

array([-0.25197354])

In [39]:
wo

-0.12669429179001424